<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Manual-fit-the-word-frequency-count" data-toc-modified-id="Manual-fit-the-word-frequency-count-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Manual fit the word frequency count</a></span><ul class="toc-item"><li><span><a href="#Translate-the-sparse-matrix-in-actual-word" data-toc-modified-id="Translate-the-sparse-matrix-in-actual-word-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Translate the sparse matrix in actual word</a></span></li></ul></li><li><span><a href="#Automate-the-extraction" data-toc-modified-id="Automate-the-extraction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Automate the extraction</a></span></li><li><span><a href="#Extract-from-all-columns" data-toc-modified-id="Extract-from-all-columns-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Extract from all columns</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_excel('interviews_20180710.xls')
df

,Name,How do you handle smart contract verification and security?,Other bounties?,Who are you and what are you working on?,What are the tools/libraries/frameworks you use?,What are your biggest frustrations?,How do you handle testing?,What tools don’t exist at the moment?,Other domain specific questions?,What was the hardest part about learning to develop with Ethereum?,What are you most excited about in the short term?,Was anything easier than expected?,Who are the other people you think we should talk to?,What are the best educational resources?,Are there any other questions we should be asking?
0,﻿Fabio Berger + Remco Bloemen,Code audits before deployment\nWant to add a f...,Rest of the bounties from Remco\nState-watchin...,0x - Decentralized exchange protocol. It is a ...,EthereumJS / truffle hardwallet provider / Typ...,Getting a simple experimental environment up i...,Testrpc has gotten better ––\nIncreases the sp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,﻿Leo Logvinov,NaN,There is a limit on local variable numbers (ha...,"Started in blockchain 2 years ago in Berlin, w...",NaN,"Event watching - unreliable, no support for ba...",NaN,Prettier type plugin for solidity. I don’t hav...,What has your experience been like working on ...,NaN,NaN,NaN,NaN,NaN,NaN
2,﻿Axel Ericsson,Human thinking and auditing.\nWill find some e...,Breaking out a javascript library for web page...,I have built 1Protocol\nIt lets smart contract...,Just web3JS and the python raiden client,NaN,"Truffle suite. Before that, built our own unit...",There is no tooling or anything related to sta...,NaN,"Solidity is changing so quickly that, as an ex...",Raiden is the golden egg in the space. The exp...,Writing smart contracts is surprisingly neat. ...,NaN,NaN,NaN
3,﻿Mike Goldin,NaN,Pretty cool to have more languages for the EVM...,Software developer at Consensys\nKnown for Tok...,"Truffle, useful as a testing framework.\nSolid...",Truffle’s debugger is a bit disappointing. Wor...,Writes tests in Javascript\nSol Coverage tool\...,Fuzz Testing and formal verification desired.,NaN,Hardest part is learning to develop in a block...,Excited for Casper\nApplications implemented i...,What’s really easy is programming money.\nOn b...,"Infura team, client development\nSpankchain - ...",Solidity Docs\nUses yellow paper gass lookup t...,NaN
4,﻿Oleksii,"We have written our own tools which tell us, f...",NaN,Started working with smart contracts in early ...,NaN,Our original vision was to do everything – tes...,NaN,NaN,What are other great tools you have used?\nEth...,NaN,NaN,NaN,NaN,NaN,NaN
5,﻿Brett Sun,Human audits only right now.\nTrailofbits is r...,NaN,Working on Aragon entirely.\nThe end goal is t...,Four categories:\nEVM and lower level stuff:\n...,My biggest general frustration comes with the ...,In addition to the above we also use solidity-...,A nice debugger! Please…\nMore infrastructure ...,NaN,NaN,NaN,NaN,NaN,Google + Github is my go to.\nNot enough has b...,NaN
6,﻿Jorge Izquierdo,NaN,Infura that is more decentralized. A full-node...,Aragon - Decentralized Governance platform\nWe...,NaN,NaN,NaN,NaN,Tell us more about getting deep into the EVM?\...,NaN,NaN,NaN,NaN,NaN,NaN
7,﻿Jack Peterson and Sparkle,Pay others to think about it - no process yet....,NaN,NaN,Back-end: Visual Studio Code with docker image...,Lack of a debugger - by far the biggest issue....,NaN,Setting breakpoints in tests!\nSalesforce Deve...,Let’s talk specifically about UI issues / scal...,What steps do I need to follow to write these ...,Raiden is coming shortly and will be very cool...,NaN,NaN,NaN,NaN
8,﻿Joey Krug,"Yeah, encourage everyone to use auditors. Lots...",NaN,Co-Chief Investment Officer at Pantera Capital...,"At Augur, stack is different as it’s a bit old...","At the end of the day, it all comes down to th...",Run against pyethereum. Test suite is written ...,"My answers changed over a time, 1 year ago it ...",Would be useful if the Ethereum project could ...,NaN,"In the short term, he’

# Preprocessing

In [3]:
df.columns = [
    'name',              # Name
    'smart_contract',    # How do you handle smart contract verif & security?
    'bounties',          # Other bounties
    'who_what',          # Who are you and what are you working on?
    'tooling',           # What are the tools/libraries/frameworks you use?
    'frustrations',      # What are your biggest frustrations?
    'testing',           # How do you handle testing?
    'missing_tools',     # What tools don’t exist at the moment?
    'domain_questions',  # Other domain specific questions?
    'hardest_part',      # What was the hardest part to develop with Ethereum?
    'excited_about',     # What are you most excited about in the short term?
    'easier_expected',   # Was anything easier than expected?
    'people_talk_to',    # Who do you think we should talk to?
    'best_resources',    # What are the best educational resources?
    'questions_to_ask'   # Are there any other questions we should be asking?
]

df.fillna('', inplace = True)

In [4]:
topics = {'solidity', 'gas', 'protocol', 'transaction', 'event', 'network', 'documentation', 'blockchain',
          'client', 'bounty', 'deployment', 'audit', 'contract', 'verification', 'token', 'governance',
         'payment', 'consensus', 'NFT', 'compiler', 'signature', 'hash', 'testing', 'wallet', 'community',
         'decentralized'}

projects = {'Ethereum', 'Geth', 'Truffle', 'Infura', 'MetaMask', 'Parity', 'Remix', 'Mythril',
            'Casper', 'Plasma', 'Status', 'Embark', 'Swarm', 'eWASM', 'Sharding', 'State channels',
           'Gitcoin', 'Augur', 'Vyper', 'Bamboo'}

# Manual fit the word frequency count

In [5]:
counter = CountVectorizer(vocabulary=topics)

In [6]:
smart_contract = counter.fit_transform(df['smart_contract'])
print(smart_contract)

  (0, 1)	1
  (0, 10)	2
  (0, 24)	1
  (2, 1)	1
  (4, 8)	1
  (4, 13)	1
  (7, 24)	1
  (8, 8)	1
  (9, 1)	1
  (16, 1)	1
  (16, 3)	2
  (16, 5)	2
  (20, 1)	1
  (20, 8)	1
  (22, 20)	2
  (22, 21)	1
  (33, 2)	1
  (33, 8)	3
  (33, 9)	1
  (33, 14)	1
  (33, 24)	2
  (35, 21)	1
  (37, 5)	1
  (37, 20)	1
  (38, 16)	1
  :	:
  (53, 3)	2
  (53, 24)	1
  (55, 2)	2
  (61, 10)	1
  (61, 11)	1
  (63, 1)	1
  (63, 8)	1
  (66, 24)	1
  (73, 1)	1
  (73, 20)	1
  (73, 24)	2
  (75, 6)	1
  (75, 20)	1
  (77, 1)	1
  (77, 24)	1
  (80, 1)	1
  (80, 3)	1
  (85, 1)	1
  (85, 2)	1
  (85, 5)	1
  (85, 13)	1
  (86, 1)	1
  (103, 8)	2
  (103, 18)	2
  (103, 24)	1


## Translate the sparse matrix in actual word

In [7]:
for row in smart_contract[0:3]:
    d = dict(zip(counter.get_feature_names(),
                 row.toarray().ravel()
            ))
    print(d)

{'NFT': 0, 'audit': 1, 'blockchain': 0, 'bounty': 0, 'client': 0, 'community': 0, 'compiler': 0, 'consensus': 0, 'contract': 0, 'decentralized': 0, 'deployment': 2, 'documentation': 0, 'event': 0, 'gas': 0, 'governance': 0, 'hash': 0, 'network': 0, 'payment': 0, 'protocol': 0, 'signature': 0, 'solidity': 0, 'testing': 0, 'token': 0, 'transaction': 0, 'verification': 1, 'wallet': 0}
{'NFT': 0, 'audit': 0, 'blockchain': 0, 'bounty': 0, 'client': 0, 'community': 0, 'compiler': 0, 'consensus': 0, 'contract': 0, 'decentralized': 0, 'deployment': 0, 'documentation': 0, 'event': 0, 'gas': 0, 'governance': 0, 'hash': 0, 'network': 0, 'payment': 0, 'protocol': 0, 'signature': 0, 'solidity': 0, 'testing': 0, 'token': 0, 'transaction': 0, 'verification': 0, 'wallet': 0}
{'NFT': 0, 'audit': 1, 'blockchain': 0, 'bounty': 0, 'client': 0, 'community': 0, 'compiler': 0, 'consensus': 0, 'contract': 0, 'decentralized': 0, 'deployment': 0, 'documentation': 0, 'event': 0, 'gas': 0, 'governance': 0, 'hash'

In [8]:
df['smart_contract'][0]
# Notice that audits (plural) is not taken into account
# We need a proper lemmatizer

'Code audits before deployment\nWant to add a formal verification tool to our deployment tool.\nBy analyzing the code, you can find patterns.\nI manually did an audit in v0 of 0x:\nMake sure all functions have a “Read -> Write -> External Call” pattern, so you don’t have any race conditions.'

In [9]:
# Cleanup
del counter
del smart_contract

# Automate the extraction

In [10]:
count_topics = CountVectorizer(vocabulary=topics)
count_projects = CountVectorizer(vocabulary=projects)

In [11]:
def dictFreq(df, field):
    counts_t = count_topics.transform(df[field])
    counts_p = count_projects.transform(df[field])
    
    result = pd.Series(
        {
            'topics': dict(zip(count_topics.get_feature_names(), counts_t[i].toarray().ravel())),
            'projects': dict(zip(count_projects.get_feature_names(), counts_p[i].toarray().ravel()))
        } for i in range(df.index.__len__())
    )
    return result

In [12]:
dictFreq(df, 'smart_contract')

0      {'topics': {'NFT': 0, 'audit': 1, 'blockchain'...
1      {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
2      {'topics': {'NFT': 0, 'audit': 1, 'blockchain'...
3      {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
4      {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
5      {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
6      {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
7      {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
8      {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
9      {'topics': {'NFT': 0, 'audit': 1, 'blockchain'...
10     {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
11     {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
12     {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
13     {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
14     {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
15     {'topics': {'NFT': 0, 'audit': 0, 'blockchain'...
16     {'topics': {'NFT': 0, 'audit': 1, 'blockchain'...
17     {'topics': {'NFT': 0, 'a

# Extract from all columns

In [13]:
cols = [
    'smart_contract',    # How do you handle smart contract verif & security?
    'bounties',          # Other bounties
    'who_what',          # Who are you and what are you working on?
    'tooling',           # What are the tools/libraries/frameworks you use?
    'frustrations',      # What are your biggest frustrations?
    'testing',           # How do you handle testing?
    'missing_tools',     # What tools don’t exist at the moment?
    'domain_questions',  # Other domain specific questions?
    'hardest_part',      # What was the hardest part to develop with Ethereum?
    'excited_about',     # What are you most excited about in the short term?
    'easier_expected',   # Was anything easier than expected?
    'people_talk_to',    # Who do you think we should talk to?
    'best_resources',    # What are the best educational resources?
    'questions_to_ask'   # Are there any other questions we should be asking?
]

In [14]:
for col in cols:
    df[col + '_wordcount'] = dictFreq(df, col)

In [15]:
df

,name,smart_contract,bounties,who_what,tooling,frustrations,testing,missing_tools,domain_questions,hardest_part,...,frustrations_wordcount,testing_wordcount,missing_tools_wordcount,domain_questions_wordcount,hardest_part_wordcount,excited_about_wordcount,easier_expected_wordcount,people_talk_to_wordcount,best_resources_wordcount,questions_to_ask_wordcount
0,﻿Fabio Berger + Remco Bloemen,Code audits before deployment\nWant to add a f...,Rest of the bounties from Remco\nState-watchin...,0x - Decentralized exchange protocol. It is a ...,EthereumJS / truffle hardwallet provider / Typ...,Getting a simple experimental environment up i...,Testrpc has gotten better ––\nIncreases the sp...,,,,...,"{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'..."
1,﻿Leo Logvinov,,There is a limit on local variable numbers (ha...,"Started in blockchain 2 years ago in Berlin, w...",,"Event watching - unreliable, no support for ba...",,Prettier type plugin for solidity. I don’t hav...,What has your experience been like working on ...,,...,"{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'..."
2,﻿Axel Ericsson,Human thinking and auditing.\nWill find some e...,Breaking out a javascript library for web page...,I have built 1Protocol\nIt lets smart contract...,Just web3JS and the python raiden client,,"Truffle suite. Before that, built our own unit...",There is no tooling or anything related to sta...,,"Solidity is changing so quickly that, as an ex...",...,"{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'..."
3,﻿Mike Goldin,,Pretty cool to have more languages for the EVM...,Software developer at Consensys\nKnown for Tok...,"Truffle, useful as a testing framework.\nSolid...",Truffle’s debugger is a bit disappointing. Wor...,Writes tests in Javascript\nSol Coverage tool\...,Fuzz Testing and formal verification desired.,,Hardest part is learning to develop in a block...,...,"{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'...","{'topics': {'NFT': 0, 'audit': 0, 'blockchain'..."
4,﻿Oleksii,"We have written our own tools which tell us, f...",,Started working with smart contracts in early ...,,Our original vision was to do everything – tes...,,,What are other great tools you have used?

In [16]:
df.to_json('interviews_wordcount.json', orient = 'index')